In [1]:
import libspn as spn
import tensorflow as tf

# Generate Dense SPN with Random Weights

In [2]:
iv=spn.IVs(num_vars=3, num_vals=2)
gen=spn.DenseSPNGenerator(num_decomps=1, num_subsets=3, num_mixtures=2)
root=gen.generate(iv)
spn.generate_weights(root, init_value=tf.initializers.random_uniform())

In [3]:
spn.display_spn_graph(root, skip_params=False)

# Save With Uninitialized Weights

In [4]:
spn.JSONSaver('/home/czikus/test_uninit.spn', pretty=True).save(root)

# Initialize and Save With Initialized Weights

In [5]:
init=spn.initialize_weights(root)
value = root.get_value()
with tf.Session() as sess:
    sess.run(init)
    first_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    spn.JSONSaver('/home/czikus/test_init.spn').save(root)

[[ 1.        ]
 [ 0.0619872 ]
 [ 0.19847235]]


# Load and Run the SPN With Uninitialzed Weights

In [6]:
loader=spn.JSONLoader('/home/czikus/test_uninit.spn')
root=loader.load()
print(root)

Sum_1


In [7]:
spn.display_spn_graph(root, skip_params=False)

In [8]:
init=spn.initialize_weights(root)
value = root.get_value()
iv = loader.find_node("IVs")
with tf.Session() as sess:
    sess.run(init)
    second_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    print(second_value)

[[ 1.        ]
 [ 0.0619872 ]
 [ 0.19847235]]
[[ 1.        ]
 [ 0.07838841]
 [ 0.13082872]]


We see that the weights of the loaded SPN were initialized after loading to random uniform distribution, since they were not initialized before saving! Therefore, second_value differs from first_value.

# Load and Run the SPN With Initialzed Weights

In [9]:
loader=spn.JSONLoader('/home/czikus/test_init.spn')
root=loader.load()
print(root)

Sum_2


In [10]:
spn.display_spn_graph(root, skip_params=False)

In [11]:
init=spn.initialize_weights(root)
value = root.get_value()
iv = loader.find_node("IVs")
with tf.Session() as sess:
    sess.run(init)
    third_value = sess.run(value, feed_dict={iv: [[-1, -1, -1],
                                                  [0,0,0],
                                                  [1,1,1]]})
    print(first_value)
    print(third_value)

[[ 1.        ]
 [ 0.0619872 ]
 [ 0.19847235]]
[[ 1.        ]
 [ 0.0619872 ]
 [ 0.19847235]]


We see that now the variables are initialized to the values to which they were initialized before. Therefore, we get the same result.